In [ ]:
#Basic imports
import math
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join

# Tracking msg 

In [ ]:
#Open files
trackFile = open('/home/sabrine/notebook/messages/trackMe.txt', 'r') 

In [ ]:
# Split text file in order to extract different field
msg = trackFile.read().split('header: \n')[1:]

In [ ]:
# Process thet tracking information
# Create different Series for the different information to easly process then later
number = []
seqs = []
secs = []
nsecs = []

# May be several id, height, distance, bounding box per sequence
ids_ = []
heights = []
distances = []
xs_bb = []
ys_bb = []
widths_bb = []
heights_bb = []

In [ ]:
len(msg)

In [ ]:
# Fill the Serie using appropriate regex to extract info
for i in range(len(msg)):
    # there should be one seq, sec, nsec and tracking information by track msg
    seq = re.search('[0-9]+',re.search('seq: [0-9]+', msg[i]).group(0)).group(0)
    sec = re.search('[0-9]+',re.search('stamp: \n    secs: [0-9]+', msg[i]).group(0)).group(0)
    nsec = re.search('[0-9]+',re.search('nsecs:\s+[0-9]+', msg[i]).group(0)).group(0)
    track = re.search('tracks:(?s).*', msg[i]).group().replace('tracks: ', '')
        
    # if severalAdvSearch('id: [0-9]+\n', track) is nan create a ligne with no trac info otherwise create a line with track info
    if not re.search('id: [0-9]+\n', track):
        # create row without track info
        print('is nan')
        number.append(i)
        seqs.append(seq)
        secs.append(sec)
        nsecs.append(nsec)
        ids_.append(0)
        heights.append('')
        distances.append('')
        xs_bb.append('')
        ys_bb.append('')
        widths_bb.append('')
        heights_bb.append('')
    else:
        #create rows with track info
        
        
        patternid = re.compile(r'(id: )([0-9]+)(\n)')
        patternheight = re.compile(r'(height: )([0-9]+\.[0-9]+)')
        patterndistance = re.compile(r'(distance: )([0-9]+\.[0-9]+)')
        patternxs = re.compile(r'(x: )([0-9]+\.[0-9]+)')
        patternys = re.compile(r'(y: )(-)?([0-9]+\.[0-9]+)')
        patternwidth = re.compile(r'(width: )([0-9]+)(\n)')
        patternheightbb = re.compile(r'(height: )([0-9]+)(\n)')
        
        #print('is not nan')
        #print('type', type(severalAdvSearch('id: [0-9]+\n', track)))
        
        for (letters, numbers, newline) in re.findall(patternid, track):
            number.append(i)
            seqs.append(seq)
            secs.append(sec)
            nsecs.append(nsec)
            ids_.append(numbers)
            print letters, '*', numbers
                        
        for (letters, numbers) in re.findall(patternheight, track):
            heights.append(numbers)
            print letters, '*', numbers
    
        for (letters, numbers) in re.findall(patterndistance, track):
            distances.append(numbers)
            print letters, '*', numbers
            
        for (letters, numbers) in re.findall(patternxs, track):
            xs_bb.append(numbers)
            print letters, '*', numbers
            
        for (letters, sign, numbers) in re.findall(patternys, track):
            ys_bb.append(numbers)
            print letters, '*', numbers
           
        for (letters, numbers, newline) in re.findall(patternwidth, track):
            widths_bb.append(numbers)
            print letters, '*', numbers
            
        for (letters, numbers, newline) in re.findall(patternheightbb, track):
            heights_bb.append(numbers)
            print letters + 'bb', '*', numbers    
        #for g in severalAdvSearch('id: [0-9]+\n', track):
            #id_ = advSearch( '[0-9]+', g)
            #print ('yo', id_)
            #break

In [ ]:
tra = pd.DataFrame(columns=['msgNumber', 'seq', 'secs', 'nsecs','id_', 'height', 'distance',
                            'x_bb', 'y_bb', 'width_bb', 'height_bb', 'time', 'frameMatch'])

In [ ]:
tra.msgNumber = number
tra.seq = seqs
tra.secs = secs
tra.nsecs = nsecs
tra.id_ = ids_
tra.height = heights
tra.distance = distances
tra.x_bb = xs_bb
tra.y_bb = ys_bb
tra.width_bb = widths_bb
tra.height_bb = heights_bb

In [ ]:
# Recreate the time of the detection in order to compare it to the frames' file name
timeDetec = (tra.secs + tra.nsecs).apply(pd.to_numeric).values.T.tolist()
tra.time = timeDetec

In [ ]:
tra.head()

In [ ]:
tra.id_ = tra.id_.astype(int)

In [ ]:
tra.msgNumber

In [ ]:
tri = tra.set_index(['msgNumber','id_'])

In [ ]:
tri.index.is_unique

# RGB images

In [ ]:
# List all file from frames folder
mypath = "/home/sabrine/notebook/framesRGB"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# Extract time from files' name
time_ = []
for t in onlyfiles:
    time_.append(int(re.search('^[0-9]*', t).group()))

In [ ]:
len(time_)

# Synchronization

In [ ]:
time_

In [ ]:
frame = []
i = 0
for msgNb, new_df in tri.groupby(level=0):
    for j in range(len(new_df)):
        frame.append(time_[i])
    i = i + 1

In [ ]:
frame

In [ ]:
len(frame)

In [ ]:
tri.frameMatch = frame

In [ ]:
tri.head(30)

In [ ]:
tra.head()

In [ ]:
tra.to_pickle("/home/sabrine/social_tracking/syncPresentation")